In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(0)
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/polynomial_train.csv",index_col = 0)

def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative (x):
    return sigmoid(x)*(1-sigmoid(x))

def R2(out,y):
    return 1 - (np.sum((out-y)**2))/(np.sum((y - np.mean(y))**2))


class Layer():
    def __init__(self,n_inputs,n_neurons):
        self.weights = np.random.randn(n_inputs,n_neurons)
        self.bias = np.random.randn(1,n_neurons)
    def forward(self,inputs):
        self.outputs = inputs @ self.weights + self.bias
        
    def backward(self,error,weights):
        self.error = ( error @ np.transpose(weights) ) * sigmoid_derivative(self.outputs)

    def set_error(self,error):
        self.error = error

    def fill():
        pass
 
def forward_propagate(x,layers,act):
    
    n = len(layers)-1
    for i in range(n):
        
        if i == 0 :
            layers[0].forward(x)
            act[0] = sigmoid(layers[0].outputs)
        else:
            layers[i].forward(act[i-1])
            act[i] = sigmoid(layers[i].outputs)

    layers[n].forward(act[n-1])
    act[n] = (layers[n].outputs)

    return layers,act

def backward_propagate(x,y,learning_rate,layers,act):
    size = act[-1].shape[0]
    n = len(layers) -1
    error = -(act[-1]-y)

    for i in range(n,0,-1):
        if i == (n):
            layers[i].set_error(error)
            Jw = np.array(act[i-1]).T @ layers[i].error 
            Jb = np.sum(layers[i].error,axis = 0)

        else :
            layers[i].backward(layers[i+1].error,layers[i+1].weights)
            Jw = np.array(act[i-1]).T @ layers[i].error
            Jb = np.sum(layers[i].error,axis = 0)

        layers[i].weights += learning_rate/size*Jw
        layers[i].bias += learning_rate/size*Jb

    layers[0].backward(layers[1].error,layers[1].weights)
    Jw = np.array(x).T @ layers[0].error
    Jb = np.sum(layers[0].error,axis = 0)

    layers[0].weights += learning_rate/size*Jw
    layers[0].bias += learning_rate/size*Jb

    return layers


In [9]:
train_data = df_train.iloc[:45000,:-1].values
test_data = df_train.iloc[45000:,:-1].values
label_train = df_train.iloc[:45000,-1].to_numpy()
label_test = df_train.iloc[45000:,-1].to_numpy()

x = (train_data - np.mean(train_data))/np.std(train_data)
x_test = (test_data - np.mean(train_data))/np.std(train_data)
y = (label_train - np.mean(label_train))/np.std(label_train)
y_test = (label_test - np.mean(label_train))/np.std(label_train)

y = y.reshape([label_train.shape[0],1])
y_test = y_test.reshape([label_test.shape[0],1])

n = 2 # no of hidden layers
m = 56 # no of neurons
layers , act = [1]*(n+1),[1]*(n+1)

for i in range(n):
    if i == 0 :
        layers[0] = Layer(3,m)
        layers[0].forward(x)
        act[0] = sigmoid(layers[0].outputs)
    else:
        layers[i] = Layer(m,m)
        layers[i].forward(act[i-1])
        act[i] = sigmoid(layers[i].outputs)
layers[n] = Layer(m,1)
layers[n].forward(act[n-1])
act[n] = (layers[n].outputs)

layers[0].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpw1.npy")
layers[0].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpb1.npy")
layers[1].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpw2.npy")
layers[1].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpb2.npy")
layers[2].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpw3.npy")
layers[2].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpb3.npy")



In [10]:
learning_rate=0.01
n_iterations = 30
start = 0
end = x.shape[0]
step = x.shape[0]//10

for i in range(n_iterations):
    for k in range(start,end,step):

        layers,act = forward_propagate(x[k:k+step,:],layers,act)
        layers = backward_propagate(x[k:k+step,:],y[k:k+step,0].reshape([step,1]),learning_rate,layers,act)
        layers,act = forward_propagate(x[k:k+step,:],layers,act)

        loss = np.mean((act[-1] - y[k:k+step,0].reshape([step,1]))**2)
        if ((i+1) % 10 == 0):
            print("After {} iterations : ".format(i+1))
            print("loss = ",loss)
            layers_test , act_test = forward_propagate(x_test,layers,act)
            score = R2(act_test[-1],y_test)
            print(score*100)

After 10 iterations : 
loss =  0.00015788673405420806
99.94174482375942
After 10 iterations : 
loss =  0.0004183937069921802
99.9422425572986
After 10 iterations : 
loss =  0.0005731404941801605
99.94243140479412
After 10 iterations : 
loss =  0.00019901376531515934
99.94257759179335
After 10 iterations : 
loss =  0.0005222090006915669
99.94268447549034
After 10 iterations : 
loss =  0.00026876126090838475
99.9426199492613
After 10 iterations : 
loss =  0.016703634602640204
99.94105718344937
After 10 iterations : 
loss =  0.0008218922701774235
99.9407927892289
After 10 iterations : 
loss =  0.000267871138151154
99.9408873232942
After 10 iterations : 
loss =  0.0006482094158221872
99.94179059536172
After 20 iterations : 
loss =  0.00015788862939173794
99.94175034636643
After 20 iterations : 
loss =  0.00041842282287712587
99.94224797106523
After 20 iterations : 
loss =  0.0005731345722543544
99.94243685097699
After 20 iterations : 
loss =  0.00019895370489688677
99.94258289945536
After 

In [ ]:
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpw1.npy",layers[0].weights)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpb1.npy",layers[0].bias)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpw2.npy",layers[1].weights)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpb2.npy",layers[1].bias)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpw3.npy",layers[2].weights)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnpb3.npy",layers[2].bias)


In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/polynomial_test_data.csv")
X_test = df_test.iloc[:,1:].values
ids = df_test.iloc[:,0].values

Test_data = (X_test - np.mean(train_data))/np.std(train_data)
layers_test,act_test = forward_propagate(Test_data,layers,act)
Y_pred = layers_test[-1].outputs
Y_res = Y_pred * np.std(label_train,axis = 0) + np.mean(label_train,axis = 0)

res = pd.DataFrame([ids,Y_res],index = ["Ids","Prediction"]).T
print(res)
res.to_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Poly/nnp_result.csv")


         Ids             Prediction
0      78886    [1685.377375445867]
1      35987    [261700.8799403772]
2      63576   [3337.9071319623654]
3      86537  [-2115210.2051618095]
4      45235  [-37546.270709370605]
...      ...                    ...
19995  42763  [-14718.868539003783]
19996  52748   [-5277.835828283631]
19997  42159   [-8328.553595482274]
19998  22095  [-1704.4829518548722]
19999  80712   [-81518.75964029648]

[20000 rows x 2 columns]
